In [9]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
%matplotlib inline
import scipy as sp
import numpy as np
import pandas as pd
# sometimes order maters on these
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import oBehave.plotting_stuff as obps
import oBehave.helperfunctions as obhf
import oBehave.neural_analysis as obna
import oBehave.behavior_anaylsis as obba
import sklearn.decomposition as decomp


from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
from visual_behavior.ophys.response_analysis.utilities import get_trace_around_timepoint,get_nearest_frame
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis

experiment_id = 639438856


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Load all the data we will want for this
manifest = obhf.load_manifest()
obba.includeNovelSession(manifest)
dataset= VisualBehaviorOphysDataset(experiment_id=experiment_id, cache_dir=obhf.drive_path)
#analysis = ResponseAnalysis(dataset)

In [ ]:
manifest.first_session

In [ ]:
def compute_lifetime_sparseness(image_responses):
    # image responses should be an array of the trial averaged responses to each image
    # sparseness = 1-(sum of trial averaged responses to images / N)squared / (sum of (squared mean responses / n)) / (1-(1/N))
    # N = number of images
    # Modified shamelessly from allen institute code
    N = float(len(image_responses))
    ls = ((1-(1/N) * ((np.power(np.abs(image_responses).sum(axis=0),2)) / (np.power(image_responses,2).sum(axis=0)))) / (1-(1/N)))
    return ls


keylist = ['experiment_id','cell_indices','cell_specimen_ids','session_type','cre_line',
           'SparAve','Spar1','Spar2','Spar3','Spar4','Spar5','Shit','Smiss']
cr = {k:[] for k in keylist}
for ex,row in manifest.iterrows():
    if not row['first_session']:
        continue
    print(row['experiment_id'])
    dataset= VisualBehaviorOphysDataset(experiment_id=row['experiment_id'], cache_dir=obhf.drive_path)
    analysis = ResponseAnalysis(dataset)
    FR = obba.includeAddRepeats(analysis.flash_response_df,dataset.all_trials)
    unique_cells = np.unique(FR['cell'])
    unique_images = np.unique(FR['image_name'])
    for ii in unique_cells:
        cr['experiment_id'].append(row['experiment_id'])
        cr['cell_indices'].append(dataset.cell_indices[ii])
        cr['cell_specimen_ids'].append(dataset.cell_specimen_ids[ii])
        cr['cre_line'].append(row['cre_line'])
        cr['session_type'].append(row['session_type'])
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['SparAve'].append(compute_lifetime_sparseness(im_means))
        for rr in [1,2,3,4,5]:
            im_means = [FR[(FR['image_name']==image)&(FR['cell']==ii)&(FR['repeats']==rr)].mean_response.mean() for image in np.unique(FR['image_name'])]
            cr['Spar'+str(rr)].append(compute_lifetime_sparseness(im_means))
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)&(FR['repeats']==1)&(FR['response_type']=='HIT')].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['Shit'].append(compute_lifetime_sparseness(im_means))
        im_means = [FR[(FR['image_name']==image)&( FR['cell']==ii)&(FR['repeats']==1)&(FR['response_type']=='MISS')].mean_response.mean() for image in np.unique(FR['image_name'])]
        cr['Smiss'].append(compute_lifetime_sparseness(im_means))    
CellResp = pd.DataFrame(cr)

644942849
loading trial response dataframe
loading flash response dataframe
645035903
loading trial response dataframe
loading flash response dataframe
645086795
loading trial response dataframe
loading flash response dataframe
645362806
loading trial response dataframe
loading flash response dataframe
646922970
loading trial response dataframe
loading flash response dataframe
647108734
loading trial response dataframe
loading flash response dataframe
647551128
loading trial response dataframe
loading flash response dataframe
647887770
loading trial response dataframe
loading flash response dataframe
652844352
loading trial response dataframe
loading flash response dataframe
653053906
loading trial response dataframe
loading flash response dataframe
653123781
loading trial response dataframe
loading flash response dataframe
639253368
loading trial response dataframe
loading flash response dataframe
639438856
loading trial response dataframe
loading flash response dataframe
639769395
lo

In [ ]:
671152642
CellResp.to_pickle('./popMeanResponseSelectivity.pkl')
CellResp.to_hdf('./popMeanResponseSelectivity.hdf5')

In [ ]:
bins = np.linspace(0,1,20)
plt.hist(cr['Spar1'],bins = bins,alpha = .5)
plt.hist(cr['Shit'],bins = bins,alpha = .5)
plt.hist(cr['Smiss'],bins = bins,alpha = .5)
sp.stats.ks_2samp(cr['Shit'],cr['Smiss'])

In [ ]:
bins = np.linspace(0,1,20)
plt.hist(cr['Spar1'],bins = bins,alpha = .5)
plt.hist(cr['Spar2'],bins = bins,alpha = .5)
plt.hist(cr['Spar3'],bins = bins,alpha = .5)
print(sp.stats.ks_2samp(cr['Spar1'],cr['Spar2']))
print(sp.stats.ks_2samp(cr['Spar1'],cr['Spar3']))
print(sp.stats.ks_2samp(cr['Spar2'],cr['Spar3']))